In [ ]:
import pandas as pd
import numpy as np
from nilearn import datasets
from nilearn import maskers
import matplotlib.pyplot as plt

from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure

import networkx as nx
from scipy.spatial.distance import cdist

In [ ]:
#Load CSV file
adhd_filename = pd.read_csv("E:/ADHD200/path_with_label.csv")
adhd_filename

In [ ]:
adhd_filename = pd.read_csv("E:/ADHD200/path_label_motion_spatial.csv")
adhd_filename

In [ ]:
adhd_pos=[]
adhd_neg=[]

for i in range(len(adhd_filename)):
    if(adhd_filename['Label'][i]==0):
        adhd_neg.append(adhd_filename['motion_spatial'][i])
    else:
        adhd_pos.append(adhd_filename['motion_spatial'][i])

In [ ]:
label = adhd_filename['Label']
label_numpy = label.to_numpy()
label_numpy

In [ ]:
np.save("E:/ADHD200/label_905.npy",label_numpy)

In [ ]:
adhd_path = adhd_filename['path'].tolist()
adhd_path
len(adhd_path)

In [ ]:
adhd_motion_spatial = adhd_filename['motion_spatial'].tolist()
adhd_motion_spatial

# ICA vased component decide and ROI generation

In [ ]:
from nilearn.image import resample_img
from nilearn.masking import compute_multi_epi_mask
from nilearn.decomposition import CanICA

# Resample all images to a common FOV
reference_img = resample_img(adhd_motion_spatial[0], target_affine=None, target_shape=None)
reference_img_shape = reference_img.shape[:3]  # get the first three dimensions
reference_img_affine = reference_img.affine
count=0;
adhd_resampled = []
for img in adhd_motion_spatial[0:150]:
    img_resampled = resample_img(img, target_affine=reference_img.affine, target_shape=reference_img_shape)
    adhd_resampled.append(img_resampled)
    count+=1
    print(count)
    


# Compute a common mask for all resampled images
mask_img = compute_multi_epi_mask(adhd_resampled, threshold=0.3)

# Fit the ICA model using the resampled images and the common mask
# canica = CanICA(n_components=20, mask=mask_img, smoothing_fwhm=6., random_state=0)
canica = CanICA(
    n_components=20,
    memory="nilearn_cache",
    memory_level=2,
    verbose=10,
    mask_strategy="whole-brain-template",
    mask=mask_img, smoothing_fwhm=6,
    random_state=0,
)
canica.fit(adhd_resampled)

In [ ]:
canica_components_img = canica.components_img_
# components_img is a Nifti Image object, and can be saved to a file with
# the following line:
canica_components_img.to_filename('E:/ADHD200/canica_resting_state_905.nii.gz')

In [ ]:
import numpy as np
import nibabel as nib

# load the .nii.gz file
canica_components_img = nib.load('E:/ADHD200/canica_resting_state.nii.gz')

# get the data as a numpy array
canica_components_numpy= canica_components_img.get_fdata()

# Print all ICA components

In [ ]:
from nilearn.plotting import plot_prob_atlas

# Plot all ICA components together
plot_prob_atlas(canica_components_img, title='All ICA components')

# ROI extraction

In [ ]:
from nilearn.regions import RegionExtractor
from nilearn import plotting
extractor = RegionExtractor(canica_components_img , threshold=0.5,
                            thresholding_strategy='ratio_n_voxels',
                            extractor='local_regions',
                            standardize=True, min_region_size=1350)
# Just call fit() to process for regions extraction
extractor.fit()
# Extracted regions are stored in regions_img_
regions_extracted_img = extractor.regions_img_
# Each region index is stored in index_
regions_index = extractor.index_
# Total number of regions extracted
n_regions_extracted = regions_extracted_img.shape[-1]

# Visualization of region extraction results
# title = ('%d regions are extracted from %d components.'
#          '\nEach separate color of region indicates extracted region'
#          % (n_regions_extracted,20))
plotting.plot_prob_atlas(regions_extracted_img, view_type='filled_contours',
                         )

In [ ]:
import matplotlib.pyplot as plt
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map

# Get the number of images

# Calculate the number of rows and columns for the subplot grid
num_rows = 4  # Adjust as needed
num_cols = 5

# Create the subplot grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))  # Adjust figsize as needed

# Iterate over the images and plot them in the subplots
for i, cur_img in enumerate(iter_img(canica_components_img)):
    # Calculate the subplot index
    row_index = i // num_cols
    col_index = i % num_cols

    # Plot the image in the corresponding subplot
    ax = axes[row_index, col_index] if num_rows > 1 else axes[col_index]
    plot_stat_map(
        cur_img,
        display_mode="z",
        title=f"Comp {int(i)}",
        cut_coords=1,
        colorbar=False,
        axes=ax
    )

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

# Time series data generation

In [ ]:
from nilearn.connectome import ConnectivityMeasure
all_data=[]
for filename  in adhd_motion_spatial:
    time_series = extractor.transform(filename)
    all_data.append(time_series) 

In [ ]:
np.save("E:/ADHD200/ICA_times_series_905.npy",all_data)